In [28]:
#!c1.8
# nusc_path = '/home/jupyter/work/resources/data/nuscenes_mini'
# prediction_path = '/home/jupyter/work/resources/monolayout_vkr/models/nu_gan_mclass_static/nu_mini/output/static'
# gt_path = os.path.join(nusc_path, 'static_gt')
# nusc_version = 'v1.0-mini'

nusc_path = '/home/jupyter/work/resources/data/nuscenes'
prediction_path = '/home/jupyter/work/resources/monolayout_vkr/models/nu_gan_mclass_static/output/static'
gt_path = os.path.join(nusc_path, 'static_gt')
nusc_version = 'v1.0-trainval'

In [29]:
#!c1.8
import argparse
import os

from matplotlib import pyplot as plt

from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import transform_matrix, Quaternion

from nuscenes.map_expansion.map_api import NuScenesMap
from nuscenes.map_expansion import arcline_path_utils
from nuscenes.map_expansion.bitmap import BitMap

import numpy as np

from PIL import Image
from skimage.draw import polygon

import cv2

In [30]:
#!c1.8

data_root = nusc_path
nusc = NuScenes(version=nusc_version, dataroot=data_root, verbose=True)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 57.566 seconds.
Reverse indexing ...
Done reverse indexing in 9.7 seconds.


# With GT

In [ ]:
#!c1.8
import glob

occ_width, occ_height = 256, 256
sensor = 'CAM_FRONT'

output_dir = f'{data_root}/video_output'
os.makedirs(output_dir, exist_ok=True)

old_location = None
classes = []

# 12hz cams
fps = 5
heigth = 512

gt_class_mapping = {
    '0': 'drivable_area',
    '1': 'ped_crossing',
    '2': 'walkway'
}

cls_color_mapping = {
    '0': np.array([1., 0, 0]),
    '1': np.array([0, 1., 0]),
    '2': np.array([0, 0, 1.])
}
    
for scene in nusc.scene:
    out_path = os.path.join(output_dir, scene['token'], 'fetched.avi')
    os.makedirs(os.path.join(output_dir, scene['token']), exist_ok=True)
    
    out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'DIVX'), fps, (3 * heigth, heigth))

    cur_sample_token = scene['first_sample_token']
    
    while cur_sample_token != "":
        cur_sample = nusc.get('sample', cur_sample_token)
        
        cam_front_data = nusc.get('sample_data', cur_sample['data'][sensor])
        filename = cam_front_data['filename'].split('/')[-1]
        
        cam_img = cv2.imread(os.path.join(nusc_path, 'samples', 'CAM_FRONT', filename))
        cam_img = cv2.resize(cam_img, (heigth, heigth))
        
        if len(classes) == 0:
            classes = sorted(os.listdir(prediction_path))

        cumulative_predicted_image = 0
        for cls_ in classes:
            img_path = os.path.join(prediction_path, cls_, filename)
            img = cv2.imread(img_path.replace('jpg', 'png'))
            img = cv2.resize(img, (heigth, heigth))
            img = np.uint8(img * cls_color_mapping[cls_])
            cumulative_predicted_image += img
            
        cumulative_gt_image = 0
        for cls_ in classes:
            img_path = os.path.join(gt_path, gt_class_mapping[cls_], filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (heigth, heigth))
            img = np.uint8(img * cls_color_mapping[cls_])
            cumulative_gt_image += img 


        image = np.concatenate((cam_img, cumulative_gt_image, cumulative_predicted_image), 1) 
        out.write(image)
                
        cur_sample_token = cur_sample['next']
        
    out.release()


# With maps

In [15]:
#!c1.8
import glob

occ_width, occ_height = 256, 256
sensor = 'CAM_FRONT'

output_dir = f'{data_root}/video_output'
os.makedirs(output_dir, exist_ok=True)

old_location = None
classes = []

# 12hz cams
fps = 5
heigth = 512
    
for scene in nusc.scene:
    out_path = os.path.join(output_dir, scene['token'], 'fetched.avi')
    os.makedirs(os.path.join(output_dir, scene['token']), exist_ok=True)
    
    out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'DIVX'), fps, (3 * heigth, heigth))

    log = nusc.get('log', scene['log_token'])

    if old_location != log['location']:
        nusc_map = NuScenesMap(dataroot=data_root, map_name=log['location'])
        old_location = log['location']

    cur_sample_token = scene['first_sample_token']
    
    
    while cur_sample_token != "":
        cur_sample = nusc.get('sample', cur_sample_token)
        
        cam_front_data = nusc.get('sample_data', cur_sample['data'][sensor])
        ego_pose = nusc.get('ego_pose', cam_front_data['ego_pose_token'])
        
        filename = cam_front_data['filename'].split('/')[-1]
        
        # in meters
        scale = 32

        patch_box = [
            ego_pose['translation'][0] - scale,
            ego_pose['translation'][1] - scale,
            ego_pose['translation'][0] + scale,
            ego_pose['translation'][1] + scale
        ]
        
        bitmap = BitMap(nusc_map.dataroot, nusc_map.map_name, 'basemap')
        map_plot = nusc_map.render_map_patch(patch_box, nusc_map.non_geometric_layers, figsize=(10, 10), bitmap=bitmap)

        map_plot[0].savefig('temp.png')
        cam_img = cv2.imread(os.path.join(nusc_path, 'samples', 'CAM_FRONT', filename))
        cam_img = cv2.resize(cam_img, (heigth, heigth))
        map_img  = cv2.imread('temp.png')
        map_img = cv2.resize(map_img, (heigth, heigth))

        if len(classes) == 0:
            classes = sorted(os.listdir(prediction_path))

        cumulative_predicted_image = 0
        for cls_ in classes:
            img_path = os.path.join(prediction_path, cls_, filename)
            img = cv2.imread(img_path.replace('jpg', 'png'))
            img = cv2.resize(img, (heigth, heigth))
            cumulative_predicted_image = img 

        image = np.concatenate((cam_img, map_img, cumulative_predicted_image), 1) 
        out.write(image)
        
        cur_sample_token = cur_sample['next']
        
    out.release()


Traceback (most recent call last):
  File "/home/jupyter/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-a44faee4cc58>", line 50, in <module>
    map_plot = nusc_map.render_map_patch(patch_box, nusc_map.non_geometric_layers, figsize=(10, 10), bitmap=bitmap)
  File "/kernel/lib/python3.7/site-packages/ml_kernel/state/state.py", line 98, in __eq__
    self._unwrap()
  File "/kernel/lib/python3.7/site-packages/ml_kernel/state/state.py", line 81, in _unwrap
    self._self_state.load_lazy(self.varname)
  File "/kernel/lib/python3.7/site-packages/ml_kernel/state/state.py", line 339, in <lambda>
    self._ml_state.load_lazy = lambda var: load_lazy(self, var, branch)
  File "/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py", line 299, in _load_lazy
    reason=REASON_LAZY_LOAD,
  File "/usr/lib/python3.7/concurrent/futures/_base.py", line 430, in result
    self._co

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

/kernel/lib/python3.7/site-packages/ml_kernel/ignored_keyboard_interrupt.py:16: UserWarning: State committing stage cannot be interrupted. Please wait.
  warnings.warn(self._warn_message)
/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:817: UserWarning: The following variables cannot be serialized: out
  warnings.warn(message)


KeyboardInterrupt: 

In [ ]:
#!c1.8
